In [1]:
import json
from pathlib import Path
import csv
import sys
sys.path.append("../")
from src.extract_fields import extract_fields_from_ocr_result

In [3]:
!pip install python-dateutil


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [7]:
OCR_JSON_DIR = Path("../data/processed/ocr")     
OUT_DIR = Path("../data/processed/extracted")
OUT_DIR.mkdir(parents=True, exist_ok=True)

splits = ["train", "test"]

for split in splits:
    src_dir = OCR_JSON_DIR / split
    out_csv = OUT_DIR / f"{split}_extracted.csv"
    rows = []
    if not src_dir.exists():
        print(f"[WARN] {src_dir} not found, skipping.")
        continue
    for jfile in sorted(src_dir.glob("*.json")):
        try:
            with open(jfile, "r", encoding="utf-8") as f:
                o = json.load(f)
        except Exception as e:
            print("Failed to read", jfile, e)
            continue
        extracted = extract_fields_from_ocr_result(o)
        rows.append({
            "image": jfile.stem,
            "vendor": extracted.get("vendor"),
            "date": extracted.get("date"),
            "total": extracted.get("total")
        })
    # write CSV
    with open(out_csv, "w", newline="", encoding="utf-8") as csvf:
        writer = csv.DictWriter(csvf, fieldnames=["image","vendor","date","total"])
        writer.writeheader()
        for r in rows:
            writer.writerow(r)
    print(f"Saved extracted fields for {split} -> {out_csv}, {len(rows)} rows")

error: nothing to repeat at position 2